In [1]:
# prepare for Python version 3x features and functions
from __future__ import division, print_function

# import packages for text processing and machine learning
import pandas as pd  # DataFrame structure and operations
import numpy as np  # arrays and numerical processing
import matplotlib.pyplot as plt  # 2D plotting
import statsmodels.api as sm  # logistic regression
import statsmodels.formula.api as smf  # R-like model specification
import patsy  # translate model specification into design matrices


In [2]:
# import user-defined module
import evaluate_classifier as eval

In [4]:
bank = pd.read_csv(r"C:\Users\Zachary_B\Desktop\Dataset\2Prospective-Student-Enrolment-Form.csv")
print(bank.head())

          Submitted  First Name  Last Name              Email Address  \
0  19/10/2022 11:15   Ikeoluwa     Adesuyi  adepojuikeoluwa@gmail.com   
1  13/10/2022 15:48  Oluwaseun    Onitiri     Coleonitiri67@gmail.com   
2  06/10/2022 20:54    IREBAMI   OLASOGBA   irebamiolasogba@gmail.com   
3  05/10/2022 21:38    Temilade  Olasogba       temilade775@gmail.com   
4  02/10/2022 19:57     Isaiah   Chukunda   josephchukunda5@gmail.com   

   Phone Number Date of Birth  Gender Marital Status  \
0  7.061368e+09    19/01/1993  Female        Married   
1  8.189806e+09    26/07/2000    Male         Single   
2  8.179189e+09    23/06/2002  Female         Single   
3  9.163363e+09    17/09/1996  Female         Single   
4  9.133480e+09    21/02/2007    Male         Single   

                Employment Status Highest Educational Qualification  ...  \
0                        Employed            Bachelor&#039;s Degree  ...   
1                      Unemployed            Bachelor&#039;s Degree  ...

In [4]:
job_to_jobtype = {'admin.':'White Collar',\
    'entrepreneur':'White Collar',\
    'management':'White Collar',\
    'self-employed':'White Collar',\
    'blue-collar':'Blue Collar',\
    'services':'Blue Collar',\
    'technician':'Blue Collar'}
bank['jobtype'] = bank['job'].map(job_to_jobtype)
bank['jobtype'] = bank['jobtype'].fillna('Other/Unknown')

In [5]:
# set marital variable
marital_to_label = {'divorced':'Divorced',\
    'married':'Married',\
    'single':'Single'}
bank['marital'] = bank['marital'].map(marital_to_label)
bank['marital'] = bank['marital'].fillna('Unknown')

In [6]:
# set education variable
education_to_label = {'primary':'Primary',\
    'secondary':'Secondary',\
    'tertiary':'Tertiary'}
bank['education'] = bank['education'].map(education_to_label)
bank['education'] = bank['education'].fillna('Unknown')

In [7]:
# set no/yes variable labels
noyes_to_label = {'no':'No', 'yes':'Yes'}
bank['default'] = bank['default'].map(noyes_to_label)
bank['default'] = bank['default'].fillna('No')

In [8]:
bank['housing'] = bank['housing'].map(noyes_to_label)
bank['housing'] = bank['housing'].fillna('No')

In [9]:
bank['loan'] = bank['loan'].map(noyes_to_label)
bank['loan'] = bank['loan'].fillna('No')

In [10]:
# code response as binary variable
noyes_to_binary = {'no':0, 'yes':1}
bank['response'] = bank['response'].map(noyes_to_binary)
bank['response'] = bank['response'].fillna('No')

In [11]:
# work only with bank clients who are being approached for the first time  
filter = bank['pdays'].map(lambda d: d == -1)

In [12]:
# apply the filter and select columns needed for targeting model
bankwork = pd.DataFrame(bank[filter], columns = ['response','age','jobtype',\
    'education',  'marital', 'default', 'balance', 'housing', 'loan'])
print(bankwork.head()) 
print(bankwork.shape) 

   response  age        jobtype  education  marital default  balance housing  \
0         0   30  Other/Unknown    Primary  Married      No     1787      No   
3         0   30   White Collar   Tertiary  Married      No     1476     Yes   
4         0   59    Blue Collar  Secondary  Married      No        0     Yes   
7         0   39    Blue Collar  Secondary  Married      No      147     Yes   
8         0   41   White Collar   Tertiary  Married      No      221     Yes   

  loan  
0   No  
3  Yes  
4   No  
7   No  
8   No  
(3705, 9)


In [13]:
# examine descriptive statistics and frequency tables for variables in model
print(bankwork.describe())
print('\njobtype:\n',bankwork['jobtype'].value_counts())
print('\nmarital:\n',bankwork['marital'].value_counts())
print('\neducation:\n',bankwork['education'].value_counts())
print('\ndefault:\n',bankwork['default'].value_counts())
print('\nhousing:\n',bankwork['housing'].value_counts())
print('\nloan:\n',bankwork['loan'].value_counts())

          response          age       balance
count  3705.000000  3705.000000   3705.000000
mean      0.090958    41.083671   1374.862078
std       0.287588    10.373818   3008.524207
min       0.000000    19.000000  -3313.000000
25%       0.000000    33.000000     60.000000
50%       0.000000    39.000000    415.000000
75%       0.000000    49.000000   1412.000000
max       1.000000    87.000000  71188.000000

jobtype:
 Blue Collar      1776
White Collar     1453
Other/Unknown     476
Name: jobtype, dtype: int64

marital:
 Married     2305
Single       957
Divorced     443
Name: marital, dtype: int64

education:
 Secondary    1891
Tertiary     1084
Primary       580
Unknown       150
Name: education, dtype: int64

default:
 No     3634
Yes      71
Name: default, dtype: int64

housing:
 Yes    2043
No     1662
Name: housing, dtype: int64

loan:
 No     3113
Yes     592
Name: loan, dtype: int64


In [14]:
# examine means of continuous explanatory variables by response
print(bankwork.pivot_table(['age'], index = ['response']))
print(bankwork.pivot_table(['balance'], index = ['response']))

                age
response           
0         40.983076
1         42.089021
              balance
response             
0         1359.654097
1         1526.851632


In [15]:
# baseline response rate computed (will be used later)
filter_took_offer = bankwork['response'].map(lambda d: d == 1)
baseline_response_rate = len(bankwork[filter_took_offer]) / len(bankwork)
print('\nBaseline proportion of clients responding to offer: ',\
    round(baseline_response_rate,5), '\n')


Baseline proportion of clients responding to offer:  0.09096 



In [16]:
# examine proportion responding across levels 
# of categorical variables
print(bankwork.pivot_table(['response'], index = ['jobtype']))
print(bankwork.pivot_table(['response'], index = ['education']))
print(bankwork.pivot_table(['response'], index = ['marital']))
print(bankwork.pivot_table(['response'], index = ['default']))
print(bankwork.pivot_table(['response'], index = ['housing']))
print(bankwork.pivot_table(['response'], index = ['loan']))

               response
jobtype                
Blue Collar    0.072072
Other/Unknown  0.144958
White Collar   0.096352
           response
education          
Primary    0.082759
Secondary  0.082496
Tertiary   0.112546
Unknown    0.073333
          response
marital           
Divorced  0.126411
Married   0.073753
Single    0.115987
         response
default          
No       0.090534
Yes      0.112676
         response
housing          
No       0.116727
Yes      0.069995
      response
loan          
No    0.098619
Yes   0.050676


In [17]:
# specify model for logisitc regression
bank_spec = 'response ~ age + jobtype + education + marital +\
    default + balance + housing + loan'

In [18]:
# ----------------------------------
# fit logistic regression model 
# ----------------------------------
# convert R-like formula into design matrix needed for statsmodels        
y,x = patsy.dmatrices(bank_spec, bankwork, return_type = 'dataframe')  

In [19]:
my_logit_model = sm.Logit(y,x)
# fit the model to the full data set
my_logit_model_fit = my_logit_model.fit()
print(my_logit_model_fit.summary())

Optimization terminated successfully.
         Current function value: 0.293877
         Iterations 7
                           Logit Regression Results                           
Dep. Variable:               response   No. Observations:                 3705
Model:                          Logit   Df Residuals:                     3692
Method:                           MLE   Df Model:                           12
Date:                Fri, 30 Sep 2022   Pseudo R-squ.:                 0.03568
Time:                        19:17:46   Log-Likelihood:                -1088.8
converged:                       True   LL-Null:                       -1129.1
Covariance Type:            nonrobust   LLR p-value:                 3.223e-12
                               coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------
Intercept                   -2.3937      0.390     -6.132      0.000      -3.159

In [20]:
# predicted probability of reponding to the offer
bankwork['pred_logit_prob'] = my_logit_model_fit.predict(linear = False)

In [21]:
# map target from probability cutoff specified
def prob_to_pred(x, cutoff):
    if(x > cutoff):
        return(1)
    else:
        return(0)

In [22]:
# try cutoff set at 0.50
bankwork['pred_logit_50'] =\
    bankwork['pred_logit_prob'].\
    apply(lambda d: prob_to_pred(d, cutoff = 0.50))    
print('\nConfusion matrix for 0.50 cutoff\n',\
    pd.crosstab(bankwork.pred_logit_50, bankwork.response, margins = True))    
# cutoff 0.50 does not work for targeting... all predictions 0 or No


Confusion matrix for 0.50 cutoff
 response          0    1   All
pred_logit_50                 
0              3368  337  3705
All            3368  337  3705


In [23]:
# try cutoff set at 0.10
bankwork['pred_logit_10'] =\
    bankwork['pred_logit_prob'].\
    apply(lambda d: prob_to_pred(d, cutoff = 0.10))    
print('\nConfusion matrix for 0.10 cutoff\n',\
    pd.crosstab(bankwork.pred_logit_10, bankwork.response, margins = True)) 


Confusion matrix for 0.10 cutoff
 response          0    1   All
pred_logit_10                 
0              2262  159  2421
1              1106  178  1284
All            3368  337  3705


In [24]:
print('\n Logistic Regression Performance (0.10 cutoff)\n',\
    'Percentage of Targets Correctly Classified:',\
    100 * round(eval.evaluate_classifier(bankwork['pred_logit_10'],\
    bankwork['response'])[4], 3),'\n')


 Logistic Regression Performance (0.10 cutoff)
 Percentage of Targets Correctly Classified: 65.9 



In [27]:
# direct calculation of lift 
# decile labels from highest to lowest 
decile_label = []
for i in range(10):
    decile_label.append('Decile_'+str(10 - i))
# draws on baseline response rate computed earlier    
def lift(x):
    return(x / baseline_response_rate)
prediction_deciles = pd.qcut(bankwork.pred_logit_prob, 10, labels = decile_label)
decile_groups = bankwork.response.groupby(prediction_deciles)
print(decile_groups.mean())
lift_values = decile_groups.mean() / baseline_response_rate
print('\nLift Chart Values by Decile:\n', lift_values, '\n')

pred_logit_prob
Decile_10    0.043127
Decile_9     0.048649
Decile_8     0.053908
Decile_7     0.072973
Decile_6     0.078167
Decile_5     0.078378
Decile_4     0.110811
Decile_3     0.118598
Decile_2     0.102703
Decile_1     0.202156
Name: response, dtype: float64

Lift Chart Values by Decile:
 pred_logit_prob
Decile_10    0.474138
Decile_9     0.534846
Decile_8     0.592672
Decile_7     0.802270
Decile_6     0.859374
Decile_5     0.861697
Decile_4     1.218261
Decile_3     1.303878
Decile_2     1.129120
Decile_1     2.222520
Name: response, dtype: float64 

